Dou et al. (2020) model

Agora em Julia versão 1.5.3

This script simulates the model

Calculando a derivada fixando os parâmetros iguais aos de Dou et al. e substituindo pelo $\beta$ encontrado na última estimação com 2 clusters

In [1]:
using LinearAlgebra, Statistics

using DataFrames
using BenchmarkTools


In [2]:
using Distributed
nprocs = 2
addprocs(nprocs);

@everywhere using NBInclude
@everywhere @nbinclude("solve_model.ipynb")

#importou a função solve_tree

pids = workers()
aux = pids[nprocs]

3

In [3]:
#Vh/D, L/D, Dj/D
data1 = [0.86, 0.19, 0.8];
n1 = 49

data2 = [2.65, 0.69, 0.74];
n2 = 17

#ρ, β, c0, λj
game_parameters = [0.4, 14.769, 0.01, 0.87];

In [4]:
A2 = @spawnat aux solve_tree(data2, game_parameters)

A1 = solve_tree(data1, game_parameters);

A2 = fetch(A2);

       From worker 3:	 24.702437 seconds (84.99 M allocations: 1.936 GiB, 1.50% gc time)
27.006758 seconds (84.99 M allocations: 1.936 GiB, 1.33% gc time)


Simulation parameters

In [5]:
#θs0, θj0, μ
simulation_parameters = [37.0, 33.0, 10.453];

### Criando funções para simular o jogo

In [6]:
#cost function
function Ct(t, c0=c0, c1=c1)
    
    #didn't use D because is normalized to D == 1.0
    #cost at period t=0(index1) is 0
    if(t <= 1)
        return 0
    else
        return c0 + c1 * (t-1) #test to make index==1 be t==0
    end
end



#liquidation payoffs
function s_L(t, L, Ds, Dj, c0, c1)
    return min(L - Ct(t, c0, c1), Ds)
end


function j_L(t, L, Ds, Dj, c0, c1)
    return min(L - Ct(t, c0, c1) - s_L(t, L, Ds, Dj, c0, c1), Dj)
end


function proposal(Pkt_array, t, hkt, lmt)
    
    #pkt array será sempre do propositor, quem responder às propostas apenas olhará o seu valor de continuação
    
    return policy, payoff_prop, payment, lm_next = Pkt_array[t, hkt, lmt, [end,end-1, 1, 2]]
    
end
    

#maximum value of reorganization each period
function Vt(Vmax, ρ, t)

    if(t <=1)
        return Vmax
    else
        #(t-2) instead of (t-1) because we shifted the indexes in the game so as to include t==0 at index==1
        return ρ^(t-2) * Vmax
    end


end

# answer_liq
function answer_liq(m_L, t, hm_next, payoff_prop, L, Ds, Dj, Vmax, ρ, c0, c1)
    
    liq = (m_L(t, L, Ds, Dj, c0, c1), Vt(Vmax, ρ, t) * hm_next/100 - Ct(t, c0, c1) - payoff_prop)
    
    payoff_liq, answer = findmax(liq)
    
    return payoff_liq, answer
end

function answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)
    
    reorg_value = (payment, Cont_val[t+1, hm_next, lm_next, lk_next])

    payoff_reorg, answer = findmax(reorg_value)
    
    return payoff_reorg, answer
end

function update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
    
    if(propositor=="s")
        hst = hk_next
        lst = lk_next
        
        hjt = hm_next
        ljt = lm_next
        
    else
        hst = hm_next
        lst = lm_next
        
        hjt = hk_next
        ljt = lk_next
    end
    
    
    return hst, lst, hjt, ljt
end

        

update_beliefs (generic function with 1 method)

In [7]:
#código para tirar um draw da beta

function draw_beta(hkt, β=game_parameters[2], grid=100)

    u = rand()

    if(hkt == grid)
        
        return grid

    else
        
        x = 1.0 - exp(1.0/β * (log(1.0 - u) + β * log(1.0 - hkt/grid)))
        x = round(x * 100, digits=0)
#         return Int64(x * 100) #to convert in an integer
        return Int64(x)
        
    end
end

        

# @code_warntype draw_beta(10)



draw_beta (generic function with 3 methods)

In [8]:
function choose_parameters(propositor, hst, lst, hjt, ljt, Pst_array, Pjt_array, s_W, j_W)
    
    if(propositor=="s")
        
        Pkt_array = Pst_array
        Cont_val = j_W
        prop_index = 1
        respondent_index = 2 #índice de j, para organizar o payoff
        m_L = j_L

        hkt = hst
        lkt = lst
        
        hmt = hjt
        lmt = ljt


        lk_next = hkt

        hk_next = draw_beta(hkt)
        hm_next = draw_beta(hmt)
        
    elseif(propositor=="j")
        Pkt_array = Pjt_array
        Cont_val = s_W
        prop_index = 2
        respondent_index = 1
        m_L = s_L

        hkt = hjt
        lkt = ljt
        
        hmt = hst
        lmt = lst


        lk_next = hkt


        hk_next = draw_beta(hkt)
        hm_next = draw_beta(hmt)

    else
        
        println("error: propositor not valid")
        
    end
    
    return Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next
end




# #test
# Hs0 = 30
# Hj0 = 28


# hst = Hs0
# hjt = Hj0

# #assumindo que lower bounds nos períodos iniciais são as próprias habilidades iniciais
# lst = hst
# ljt = hjt




# choose_parameters("s", hst, lst, hjt, ljt);
    
    

In [9]:
function simulate_game(data, game_parameters, simulation_parameters, s_W=s_W, j_W=j_W, Pst_array=Pst_array, Pjt_array=Pjt_array, t=1, grid=100)
    
    
    Vmax = data[1]
    L = data[2]
    Dj = data[3]
    Ds = one(1.0) - Dj
    
    
    c0 = game_parameters[3];
    c1 = c0/30;

    ρ = game_parameters[1];
    λj = game_parameters[4];

    
    
#     hst = Hs0
#     hjt = Hj0
    
    hst = Int64(simulation_parameters[1])
    hjt = Int64(simulation_parameters[2])

    #assumindo que lower bounds nos períodos iniciais são as próprias habilidades iniciais
    lst = hst
    ljt = hjt
    
    result = zeros(5);
    
    #number of observed proposals
    observed_proposals = 0.0
    
    
    #recovering "T+1" from s_W 
    T = size(s_W,1)
    
    
    while(result[1]==0.0 && t < T)
        
        
        u = rand()

        if(u < λj)
            propositor = "j"

        else
            propositor = "s"

        end

        
        #setting the default variables according to the propositor
        Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next = choose_parameters(propositor, hst, lst, hjt, ljt, Pst_array, Pjt_array, s_W, j_W)
        
        #proposal ####
        policy, payoff_prop, payment, lm_next = proposal(Pkt_array, t, hkt, lmt)

        #lm_next is the update of the adversary's lower bound
        if(lm_next>=grid+1 || policy!= 3.0) #para evitar update de lowerbound quando proponente não propõe reorg
            lm_next = lmt
        else
            lm_next = Int64(lm_next)
        end
        
        
        
        if(policy==3.0)
            
            observed_proposals += 1.0
    
            payoff_respondent, answer = answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)


            if(answer==1.0)

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 3.0
                result[4] = t
                result[5] = observed_proposals

            else
                
                t+=1

                hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
            end

        elseif(policy==2.0)
            
            t+=1
            
            hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
            
            

        else      
            #(policy==1.0)
            
            observed_proposals += 1.0

            payoff_respondent, answer = answer_liq(m_L, t, hm_next, payoff_prop, L, Ds, Dj, Vmax, ρ, c0, c1)
            
            

            if(answer==1.0)

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 1.0
                result[4] = t
                result[5] = observed_proposals

            else

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 3.0
                result[4] = t
                result[5] = observed_proposals


            end
        end


        if(t==T)
            
            result[1] = s_W[T, 1, 1, 1]#todos os valores finais de S são iguais, então acessarei o índice 1
            result[2] = j_W[T, 1, 1, 1]
            result[3] = 1.0
            result[4] = t
            result[5] = max(observed_proposals, 1.0)
            
        end
        




    end
    
    
    return result
end

        
        
#precisa colocar uma condição IF aqui para caso o jogo vá até o último período?
    

simulate_game (generic function with 7 methods)

In [10]:
#unit testing
@btime simulate_game(data1, game_parameters, simulation_parameters, A1[1], A1[2], A1[3], A1[4])

  3.425 μs (22 allocations: 992 bytes)


5-element Array{Float64,1}:
 0.18167279813146475
 0.19889566948121437
 3.0
 2.0
 2.0

In [11]:
function loop_simulations(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    
    
    
    
    #S is the number of simulations
    #N is the number of observations
    #data1 is the center of the first cluster
    #A1 is the collection of arrays from the first cluster (s_W, j_W, Pst, Pjt)
    #n1 is the number of observations in the first cluster
    
    
    
    
    #no futuro os argumentos podem ser arrays de arrays, assim ele faz o loop para cada cluster
    Results = zeros(S, N, 7)

    #últimas entradas de Results são os valores das dívidas
    
    
    for s in 1:S
        for n in 1:N
            if(n <= n1)
                Results[s,n,1:5] .= simulate_game(data1, game_parameters, simulation_parameters, A1[1], A1[2], A1[3], A1[4])
                Results[s,n, end-1] = A1[5]
                Results[s,n, end] = A1[6]
            else
                Results[s,n,1:5] .= simulate_game(data2, game_parameters, simulation_parameters, A2[1], A2[2], A2[3], A2[4])
                Results[s,n, end-1] = A2[5]
                Results[s,n, end] = A2[6]
            end
            
        end
    end

    return Results
    
end

    
# loop_simulations(40, 75, θs0, θj0, μ, λj);

In [12]:
#unit testing
S = 40
N = 66
Results = loop_simulations(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

40×66×7 Array{Float64,3}:
[:, :, 1] =
 0.181686  0.181726  0.220502  0.181686  …  0.260307  0.260715  0.260194
 0.181826  0.181672  0.179357  0.179407     0.260261  0.260194  0.260418
 0.179431  0.181726  0.179522  0.179357     0.260522  0.803949  0.260745
 0.181869  0.179431  0.384262  0.181869     0.260418  0.260194  0.260962
 0.36066   0.179522  0.220502  0.220502     0.26054   0.779288  0.260261
 0.181672  0.337058  0.179407  0.344925  …  0.26054   0.260194  0.260194
 0.220502  0.344925  0.36066   0.179522     0.260261  0.260261  0.260307
 0.226693  0.179522  0.181867  0.352793     0.260962  0.351706  0.260307
 0.181869  0.181686  0.188853  0.181672     0.26054   0.260194  0.26054
 0.36066   0.181801  0.179431  0.181869     0.260962  0.803949  0.260418
 0.179522  0.179431  0.18171   0.179431  …  0.260715  0.260194  0.26054
 0.181726  0.179522  0.181867  0.179357     0.260003  0.260003  0.26056
 0.179357  0.18171   0.179357  0.352793     0.260194  0.26054   0.26054
 ⋮               

In [13]:
function simulate_moments(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    
    Results = loop_simulations(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

    Moments = zeros(S, 8)

    μ = simulation_parameters[3]
    for s in 1:S

        DF = DataFrame()
        DF.payoff_s = Results[s,:,1];
        DF.payoff_j = Results[s,:,2];
        DF.out = Results[s,:,3];
        DF.t = Results[s,:,4];
        DF.observed_proposals = Results[s,:,5];
        DF.Ds = Results[s,:,6];
        DF.Dj = Results[s,:,7];


        DF.outcome = ifelse.(DF.out .==3.0, "R", "L");


        #desfazendo o deslocamento no índice de t
        DF.t = DF.t .- 1.0;

        # DF.incourt = ifelse.(DF.t .> 0.0, "incourt", "precourt");

        INCOURT = filter(DF -> DF.t .> 0.0, DF);
        PRECOURT = filter(DF -> DF.t .<= 0.0, DF);

        PRECOURT_REORG = filter(PRECOURT -> PRECOURT.outcome .== "R", PRECOURT)
        INCOURT_REORG = filter(INCOURT -> INCOURT.outcome .== "R", INCOURT)

        #cálculo dos momentos
        #1. avg log number of months between observed proposals incourt

        mm1 = INCOURT
        mm1 = log.(mm1.t .* μ ./ mm1.observed_proposals)
        mm1 = mean(mm1)

        #2. fraction reorganized given that the case went into court

        mm2 = INCOURT
        mm2 = size(filter(mm2 -> mm2.outcome .== "R", mm2),1)/ size(mm2,1)

        #3. ln duration of court cases in months

        mm3 = INCOURT
        mm3 = filter(mm3 -> mm3.t .> 0.0, mm3) #removendo os casos 0 para não poluir a média
        mm3.t = mm3.t .* μ
        mm3 = mean(log.(mm3.t)) #log here uses exp as base, so it's the same as ln

        #4. fraction of cases incourt
        mm4 = size(INCOURT,1) / size(DF, 1)

        #5. avg recovery rate for senior given precourt REORGANIZATION

        #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

        mm5 = PRECOURT_REORG
        mm5.R_s = mm5.payoff_s ./ mm5.Ds
        mm5 = mean(mm5.R_s)



        #6. avg recovery rate for junior given precourt REORGANIZATION

        #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

        mm6 = PRECOURT_REORG
        mm6.R_j = mm6.payoff_j ./ mm6.Dj
        mm6 = mean(mm6.R_j)


        #7. junior avg fraction gain given incourt REORGANIZATION

        mm7 = INCOURT_REORG
        mm7 = mean(mm7.payoff_j ./ (mm7.payoff_j .+ mm7.payoff_s )) 

        #8. total recovery rate given incourt REORGANIZATION

        mm8 = INCOURT_REORG
        mm8 = mean( mm8.payoff_s .+ mm8.payoff_j)

        Moments[s,:] .= [mm1, mm2, mm3, mm4, mm5, mm6, mm7, mm8]


        #if there are no INCOURT or no PRECOURT cases, it will return NaN
        #thus, we replace it by zero
        Moments[s,:] .= replace!(Moments[s,:], NaN=> zero(0.0))

    end





    #9. slope of the regression of log(recovery rate | incourt reorg) ~ log(duration)

    #fazendo a regressão do nono momento fora do loop para poupar tempo
    #isto é, ao invés de fazer uma regressão para cada simulação,
    #faremos uma regressão só usando todas as simulações
    df = DataFrame()

    for s in 1:S
        df_result = DataFrame(Results[s,:, 1:4])
        append!(df, df_result)
    end

    rename!(df, [:x1, :x2, :x3, :x4] .=> [:payoff_s, :payoff_j, :out, :t])



    #desfazendo o deslocamento no índice de t
    df.t = df.t .- 1.0;

    #keeping only incourt reorganizations
    df = filter(df -> df.t .> 0.0, df);
    df = filter(df -> df.out .== 3.0, df)


    x = log.(df.payoff_s .+ df.payoff_j)

    X = [ones(size(x,1)) x]

    y = log.(df.t .* μ)

    if(size(df, 1) == 0)
        mm9 = zero(0.0)
    else
        mm9 = (X' * X) \ (X' * y)
        mm9 = mm9[2]
    end


    MM = vec(mean(Moments, dims=1))
    MM = vcat(MM, mm9)

    return MM

end

simulate_moments (generic function with 1 method)

In [14]:
using Random

Random.seed!(2021)

#test
S = 100
N = 66

simulated_moments = simulate_moments(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

9-element Array{Float64,1}:
  2.291160835077811
  0.9922022951305408
  2.380386670654286
  0.9463636363636363
  1.1231630937872603
  0.28273372977310646
  0.4674313769144564
  0.534988085118683
 -0.07831011314735048

Calculando a matriz de pesos ótima com os resultados dessa estimação


Two-step variance covariance estimator of W
https://notes.quantecon.org/submission/5b3db2ceb9eab00015b89f93



In [18]:
#data_moments
data_moments = [2.11, 0.843, 2.76, 0.969, 0.6235, 0.5569, 0.773, 0.497, -0.098];

In [19]:
function omega(N, data_moments, simulated_moments)
    
    error = (simulated_moments .- data_moments) ./ data_moments
    
    omega = 1/N * error * transpose(error)
    
    return omega
end

omega (generic function with 1 method)

In [20]:
W_2step = pinv(omega(N, data_moments, simulated_moments))

9×9 Array{Float64,2}:
  0.370964   0.764711  -0.594267  …   -1.70796    0.330248   -0.868094
  0.764711   1.57639   -1.22503       -3.52083    0.68078    -1.7895
 -0.594267  -1.22503    0.95199        2.73608   -0.529043    1.39065
 -0.100933  -0.208065   0.16169        0.464708  -0.0898549   0.236193
  3.4625     7.13767   -5.54678      -15.9418     3.08247    -8.10262
 -2.12709   -4.38483    3.40751   …    9.79341   -1.89363     4.97762
 -1.70796   -3.52083    2.73608        7.86369   -1.52051     3.99682
  0.330248   0.68078   -0.529043      -1.52051    0.294002   -0.772816
 -0.868094  -1.7895     1.39065        3.99682   -0.772816    2.03143

In [21]:
#pesos dados a cada um dos momentos
diag(W_2step)

9-element Array{Float64,1}:
  0.3709636778796731
  1.5763894865257808
  0.9519900604881276
  0.0274620808354169
 32.3183590186174
 12.196670280828661
  7.863686622877914
  0.2940016033191188
  2.0314304639745373

Calculando a derivada dos momentos em relação aos parâmetros

In [22]:
function simulate_new_moments(i, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    
    println("parameter ",i, game_parameters, simulation_parameters)
        
    θs0, θj0, μ = simulation_parameters;
    θs0 = Int64(round(θs0, digits=0));
    θj0 = Int64(round(θj0, digits=0));

    λj = game_parameters[4];
    new_simulated_moments = simulate_moments(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1) 
    
    return new_simulated_moments
end

simulate_new_moments (generic function with 1 method)

In [23]:
 
function derivative_moments(S, N, game_parameters, simulation_parameters, data1, data2, n1, aux)

    
    
    
    #https://en.wikibooks.org/wiki/Introduction_to_Numerical_Methods/Numerical_Differentiation
    #backward divided difference method

   


    estimated_game_parameters = game_parameters
    estimated_simulation_parameters = simulation_parameters



    A2 = @spawnat aux solve_tree(data2, game_parameters)
    A1 = solve_tree(data1, game_parameters)
    A2 = fetch(A2)
    
    
    
    simulated_moments = simulate_moments(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

    P = size(game_parameters,1)+size(simulation_parameters,1)
    M = size(simulated_moments,1)
    D = zeros(M, P)
    
    #porcentagem de variação no parâmetro
    pct = 1/100
    
    for p in P:-1:1

        if(p>size(game_parameters,1))

            simulation_parameters = copy(estimated_simulation_parameters)
            
            delta = simulation_parameters[p-size(game_parameters,1)] * pct

            simulation_parameters[p-size(game_parameters,1)] -= delta


            if(p == 5 || p ==6) #para ajustar caso seja habilidade inicial de S ou de J
                simulation_parameters[p-size(game_parameters,1)] = round(simulation_parameters[p-size(game_parameters,1)], digits=0)
            end
            
            
            #caso a variação percentual não seja suficiente para reduzir habilidades iniciais, delta será 1
            if(simulation_parameters[p-size(game_parameters,1)] == estimated_simulation_parameters[p-size(game_parameters,1)])
                delta = Int64(1)
                
                simulation_parameters[p-size(game_parameters,1)] -= delta
            end

            new_simulated_moments = simulate_new_moments(p, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

            D[:, p] = (simulated_moments - new_simulated_moments) ./ delta
            
        else

            #se for um parâmetro de jogo, precisa recalcular o jogo

            game_parameters = copy(estimated_game_parameters)
            
            delta = game_parameters[p] * pct
            game_parameters[p] -= delta

            A2 = @spawnat aux solve_tree(data2, game_parameters)
            A1 = solve_tree(data1, game_parameters)
            A2 = fetch(A2)



            new_simulated_moments = simulate_new_moments(p, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

            D[:, p] = (simulated_moments - new_simulated_moments) ./ delta


        end


    end
    
    
    
    return D
end
    

derivative_moments (generic function with 1 method)

In [24]:
Derivative = derivative_moments(S, N, game_parameters, simulation_parameters, data1, data2, n1, aux)

       From worker 3:	 23.313710 seconds (83.73 M allocations: 1.874 GiB, 0.97% gc time)
25.404923 seconds (83.73 M allocations: 1.874 GiB, 1.40% gc time)
parameter 7[0.4, 14.769, 0.01, 0.87][37.0, 33.0, 10.348469999999999]
parameter 6[0.4, 14.769, 0.01, 0.87][37.0, 32.0, 10.453]
parameter 5[0.4, 14.769, 0.01, 0.87][36.0, 33.0, 10.453]
       From worker 3:	 23.419455 seconds (83.73 M allocations: 1.874 GiB, 1.61% gc time)
25.832891 seconds (83.73 M allocations: 1.874 GiB, 1.72% gc time)
parameter 4[0.4, 14.769, 0.01, 0.8613][36.0, 33.0, 10.453]
       From worker 3:	 25.177125 seconds (83.73 M allocations: 1.874 GiB, 1.44% gc time)
27.513261 seconds (83.73 M allocations: 1.874 GiB, 1.63% gc time)
parameter 3[0.4, 14.769, 0.0099, 0.87][36.0, 33.0, 10.453]
       From worker 3:	 23.478976 seconds (83.73 M allocations: 1.874 GiB, 1.61% gc time)
25.621719 seconds (83.73 M allocations: 1.874 GiB, 1.55% gc time)
parameter 2[0.4, 14.62131, 0.01, 0.87][36.0, 33.0, 10.453]
       From worker 3

9×7 Array{Float64,2}:
  3.07826    0.111155     181.557     2.59262    …   0.00560242    0.122633
 -0.579151   0.00232621    13.0025    0.0206242      0.00234283    0.00861606
  1.6718     0.0147791     46.913     0.540388      -0.00307306    0.0858879
 -4.96212   -0.140548    -189.394    -2.43817       -0.000909091  -0.0231918
 46.5336     0.859421    1514.7      14.461         -0.0107433    -0.0457803
 18.8756     0.408077     730.316     1.23518    …  -0.0282202    -0.465375
 -1.58418   -0.013891    -116.297    -0.591068       0.00582303   -0.0141675
  0.183037   0.0161065      6.69913   0.344603       0.0105443     0.0196973
 -3.05808   -0.037862    -142.294    -1.21377       -0.000394643   0.00378559

In [25]:
Derivative[:,2]

9-element Array{Float64,1}:
  0.11115489089649357
  0.0023262116256527504
  0.014779107130531127
 -0.14054828192549165
  0.859421006386204
  0.40807658652734674
 -0.013891007533846781
  0.01610645178977582
 -0.03786198487329508

In [26]:
using DelimitedFiles

writedlm( "Derivative-nocd50_1pct.csv",  Derivative, ',')